In [ ]:
# ==========================================
# CELLULE COMPLETE : DONNÉES + EXÉCUTION ROBUSTE (OPTIMISÉE RAM)
# ==========================================
import os
import gc
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer, util, losses, evaluation, models
from transformers import AutoConfig, AutoModel, AutoTokenizer
from torch.utils.data import DataLoader

# --- 0. FONCTION DE NETTOYAGE RAM ---
def clean_memory():
    """Force le nettoyage de la mémoire RAM et GPU"""
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.ipc_collect()

# --- 1. LES DONNÉES ---
DATASET_JARGON_TRIPLETS = [
    ("Assiette", "Base de calcul de l'impôt", "Vaisselle pour manger"),
    ("Liquidation", "Calcul du montant de l'impôt exigible", "Soldes en magasin"),
    ("Recouvrement", "Action de percevoir l'impôt", "Couvrir un objet"),
    ("Role", "Titre exécutoire collectif pour l'impôt", "Personnage de cinéma"),
    ("Foyer", "Unité d'imposition (Foyer fiscal)", "Cheminée ou lieu d'habitation"),
    ("Abattement", "Déduction sur la base imposable", "Découragement ou fatigue"),
    ("Grace", "Remise gracieuse d'une dette fiscale", "Élégance ou prière"),
    ("Timbre", "Taxe payée pour une formalité", "Vignette postale de collection"),
    ("Retenue", "Somme prélevée à la source", "Punition scolaire"),
    ("Imputation", "Application d'un crédit d'impôt", "Accusation d'une faute"),
    ("Exploitant", "Professionnel agricole ou commercial", "Personne qui profite des autres"),
    ("Redevance", "Somme versée pour service public", "Loyer féodal historique"),
    ("Vacance", "État d'un logement vide", "Période de congés scolaires"),
    ("Exercice", "Période budgétaire annuelle", "Entraînement sportif"),
    ("Ventilation", "Répartition budgétaire", "Système d'aération"),
    ("Engagement", "Acte créant une dette pour l'État", "Promesse de mariage"),
    ("Liquide", "Disponibilités financières", "État de l'eau"),
    ("Balance", "État récapitulatif des comptes", "Instrument de pesée"),
    ("Provision", "Réserve pour risque", "Nourriture pour voyage"),
    ("Ecriture", "Enregistrement comptable", "Calligraphie"),
    ("Mandat", "Ordre de payer une dépense", "Durée de fonction d'un élu"),
    ("Titre", "Justificatif de recette", "Nom d'un livre"),
    ("Chapitre", "Subdivision d'un budget", "Partie d'un roman"),
    ("Amortissement", "Perte de valeur d'un bien", "Réduction des chocs"),
    ("Liasse", "Ensemble des documents fiscaux", "Paquet de billets"),
    ("Clôture", "Arrêt des comptes", "Barrière de jardin"),
    ("Immobilisation", "Bien durable de l'entité", "Plâtre médical"),
    ("Instruction", "Texte d'application d'une loi", "Éducation des enfants"),
    ("Arrêté", "Décision administrative", "Fait d'être stoppé"),
    ("Disposition", "Article d'un texte de loi", "Arrangement de meubles"),
    ("Mutation", "Changement de propriétaire", "Transformation biologique"),
    ("Minute", "Original d'un jugement", "Unité de temps"),
    ("Grosse", "Copie exécutoire d'un jugement", "Personne forte"),
    ("Saisie", "Confiscation de biens", "Frappe au clavier"),
    ("Parquet", "Ministère public", "Sol en bois"),
    ("Siège", "Adresse de la direction", "Chaise ou fauteuil"),
    ("Ressort", "Compétence géographique", "Pièce élastique"),
    ("Frais", "Dépenses professionnelles", "Température froide"),
    ("Service", "Division administrative", "Mise de table"),
    ("Corps", "Ensemble de fonctionnaires", "Partie physique"),
    ("Cadre", "Catégorie d'emploi public", "Bordure de tableau"),
    ("Mouvement", "Mutation de fonctionnaires", "Déplacement physique"),
    ("Obligation", "Titre de dette de l'État", "Devoir moral"),
    ("Action", "Part de capital", "Film d'aventure"),
    ("Trésor", "L'État banquier", "Coffre de pirate"),
    ("Bulle", "Surévaluation des actifs", "Sphère de savon"),
    ("Plafond", "Limite maximale", "Haut d'une pièce"),
    ("Plancher", "Limite minimale", "Sol d'une pièce"),
    ("Flottant", "Partie du capital en bourse", "Objet qui ne coule pas"),
    ("Masse", "Montant global (Masse salariale)", "Outil marteau"),
    ("Inflation", "Hausse des prix", "Gonflement d'un ballon"),
    ("Dépression", "Crise économique", "Maladie mentale"),
    ("Recette", "Rentrée d'argent public", "Instruction de cuisine"),
    ("Credit", "Autorisation de dépense", "Confiance"),
    ("Enveloppe", "Budget alloué", "Papier pour lettre"),
    ("Cycle", "Fluctuation économique", "Vélo"),
    ("Taux", "Pourcentage fiscal", "Animal (Taupe)")
]

DATASET_QA_TRIPLETS = [
    ("Que fait la DGFiP ?", "Elle gère la fiscalité et les finances publiques.", "Elle gère les douanes et les frontières."),
    ("C'est quoi la TICPE ?", "Taxe sur les produits énergétiques (carburants).", "Taxe sur les produits électroniques importés."),
    ("Définition de l'IS", "Impôt sur les bénéfices des sociétés.", "Impôt sur la fortune des particuliers."),
    ("Rôle de la Cour des Comptes", "Vérifier le bon emploi de l'argent public.", "Voter le budget de l'État chaque année."),
    ("C'est quoi le PAS ?", "Le Prélèvement à la Source de l'impôt revenu.", "Le Plan d'Action Sociale du ministère."),
    ("Qu'est-ce qu'un Rescrit ?", "Une garantie donnée par l'administration sur une situation.", "Une amende pour retard de paiement."),
    ("Définir la DGF", "Dotation de l'État versée aux collectivités locales.", "Dotation pour la Gestion des Forêts."),
    ("C'est quoi le Cadastre ?", "Le registre des propriétés foncières.", "Le registre des casiers judiciaires."),
    ("Qui vote le Budget ?", "Le Parlement (Assemblée et Sénat).", "Le Ministère de l'Économie."),
    ("C'est quoi la TVA ?", "Un impôt indirect sur la consommation.", "Un impôt direct sur le revenu."),
    ("Définir le PFU", "Prélèvement Forfaitaire Unique sur le capital.", "Plan de Financement Urbain."),
    ("Rôle de Tracfin", "Lutte contre le blanchiment d'argent.", "Lutte contre la fraude dans les transports."),
    ("C'est quoi une Niche fiscale ?", "Un avantage réduisant l'impôt légalement.", "Une fraude pour ne pas payer d'impôt."),
    ("Définir le Déficit public", "Quand les dépenses dépassent les recettes.", "La somme totale des dettes accumulées."),
    ("C'est quoi la CFE ?", "Cotisation Foncière des Entreprises.", "Caisse Française d'Épargne."),
    ("Rôle de la DGDDI", "Surveillance des frontières et marchandises.", "Gestion des impôts sur le revenu."),
    ("C'est quoi le Bouclier Tarifaire ?", "Plafonnement des prix de l'énergie.", "Taxe sur les importations d'acier."),
    ("Définir l'Amortissement", "Perte de valeur comptable d'un bien.", "Remboursement total d'une dette."),
    ("C'est quoi une Liasse fiscale ?", "Ensemble des déclarations de résultat.", "Un contrôle fiscal approfondi."),
    ("Définir le Recouvrement", "L'action d'encaisser les impôts.", "L'action de calculer le montant de l'impôt.")
]

from sentence_transformers import InputExample
DATASET_FT_PAIRS = [InputExample(texts=[q, a]) for q, a, _ in DATASET_QA_TRIPLETS] + \
                   [InputExample(texts=[t, d]) for t, d, _ in DATASET_JARGON_TRIPLETS[:30]]

FILENAME = "resultats_bercy_safe.csv"

# --- 2. FONCTIONS DE TEST OPTIMISÉES RAM ---
def evaluate_triplets(model, dataset):
    """Méthode 1 : Test Jargon (Triplets)"""
    anchors = [x[0] for x in dataset]
    positives = [x[1] for x in dataset]
    negatives = [x[2] for x in dataset]

    emb_A = model.encode(anchors, convert_to_tensor=True)
    emb_P = model.encode(positives, convert_to_tensor=True)
    emb_N = model.encode(negatives, convert_to_tensor=True)

    sim_AP = util.cos_sim(emb_A, emb_P).diagonal()
    sim_AN = util.cos_sim(emb_A, emb_N).diagonal()

    correct = 0
    for i in range(len(dataset)):
        if sim_AP[i] > sim_AN[i]:
            correct += 1
    return (correct / len(dataset)) * 100

def evaluate_efficiency(model, train_data):
    """
    Méthode 2 : Fine-tuning SANS COPIE (In-place)
    On modifie le modèle directement pour éviter de doubler la RAM.
    """
    val_q = ["Foncière", "Douane", "Contrôle", "Majoration"]
    val_a = ["Impôt propriétaire", "Surveillance frontières", "Vérification dossier", "Pénalité retard"]
    evaluator = evaluation.EmbeddingSimilarityEvaluator(val_q, val_a, [1.0]*4)

    # 1. Score AVANT
    score_before = evaluator(model)

    # 2. Entraînement sur CPU (Plus lent mais SÛR pour la RAM)
    # Si vous avez un gros GPU, vous pouvez tenter de retirer .cpu()
    model.cpu()

    train_dl = DataLoader(train_data, shuffle=True, batch_size=4)
    loss = losses.MultipleNegativesRankingLoss(model)

    model.fit(
        train_objectives=[(train_dl, loss)],
        epochs=1,
        show_progress_bar=False,
        use_amp=False
    )

    # 3. Score APRÈS
    score_after = evaluator(model)

    return score_before, score_after

# --- 3. LISTE DES CANDIDATS ---
CANDIDATS = [
    {"name": "Solon (Juridique)", "id": "OrdalieTech/SOLON-embeddings-large-0.1", "type": "solon", "color": "#e74c3c"},
    {"name": "Snowflake-Arctic", "id": "Snowflake/snowflake-arctic-embed-m-v2.0", "type": "snowflake", "color": "#3498db"},
    {"name": "BGE-M3", "id": "BAAI/bge-m3", "type": "standard", "color": "#2ecc71"},
    {"name": "GTE-Multilingual", "id": "Alibaba-NLP/gte-multilingual-base", "type": "trust", "color": "#f1c40f"},
    {"name": "E5-Large-Instruct", "id": "intfloat/multilingual-e5-large-instruct", "type": "standard", "color": "#9b59b6"},
    {"name": "E5-Small", "id": "intfloat/multilingual-e5-small", "type": "standard", "color": "#95a5a6"},
]

# --- 4. LOGIQUE DE REPRISE ---
deja_faits = []
if os.path.exists(FILENAME):
    df_exist = pd.read_csv(FILENAME)
    deja_faits = df_exist["Modèle"].tolist()
    print(f"🔄 Reprise de session : {len(deja_faits)} modèles déjà faits ({', '.join(deja_faits)})")
else:
    df_vide = pd.DataFrame(columns=["Modèle", "Jargon Accuracy", "Efficiency Gain", "Color"])
    df_vide.to_csv(FILENAME, index=False)

# --- 5. BOUCLE PRINCIPALE ---
for cand in CANDIDATS:
    if cand['name'] in deja_faits:
        continue

    print(f"\n⚙️  Traitement de : {cand['name']}...")
    clean_memory() # Nettoyage AVANT

    try:
        model = None
        # Chargement intelligent (Avec FP16 si possible pour diviser la RAM par 2)
        load_args = {"model_kwargs": {"torch_dtype": torch.float16}}

        if cand['type'] == 'solon':
             w = models.Transformer(cand['id'], model_args={"torch_dtype": torch.float16})
             p = models.Pooling(w.get_word_embedding_dimension(), pooling_mode='mean')
             model = SentenceTransformer(modules=[w, p])
        elif cand['type'] == 'snowflake':
             model = SentenceTransformer(cand['id'], trust_remote_code=True,
                                         model_kwargs={"use_memory_efficient_attention": False, "unpad_inputs": False, "torch_dtype": torch.float16})
        elif cand['type'] == 'trust':
             model = SentenceTransformer(cand['id'], trust_remote_code=True, **load_args)
        else:
             model = SentenceTransformer(cand['id'], **load_args)

        # Tests
        score_jargon = evaluate_triplets(model, DATASET_JARGON_TRIPLETS)
        s_bef, s_aft = evaluate_efficiency(model, DATASET_FT_PAIRS)
        gain = ((s_aft - s_bef)/s_bef)*100 if s_bef > 0 else 0

        print(f"   ✅ Jargon: {score_jargon:.1f}% | Gain: {gain:+.1f}%")

        # Sauvegarde
        new_row = pd.DataFrame([{
            "Modèle": cand['name'],
            "Jargon Accuracy": score_jargon,
            "Efficiency Gain": gain,
            "Color": cand['color']
        }])
        new_row.to_csv(FILENAME, mode='a', header=False, index=False)
        print(f"   💾 Sauvegardé dans {FILENAME}")

    except Exception as e:
        print(f"   ❌ Erreur sur {cand['name']} : {e}")

    # Nettoyage APRES
    del model
    clean_memory()

print("\n🎉 Tous les modèles sont traités !")

🔄 Reprise de session : 0 modèles déjà faits ()

⚙️  Traitement de : Solon (Juridique)...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
`torch_dtype` is deprecated! Use `dtype` instead!
/usr/local/lib/python3.12/dist-packages/sentence_transformers/evaluation/EmbeddingSimilarityEvaluator.py:195: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  eval_pearson, _ = pearsonr(labels, scores)
/usr/local/lib/python3.12/dist-packages/sentence_transformers/evaluation/EmbeddingSimilarityEvaluator.py:196: ConstantInputWarning: An input array is constant; the correlation coeffic

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

wandb: Currently logged in as: naveen06 (naveen06-ece-paris-fencing) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
